## Task 4

You've been flown to their headquarters in Ulsan, South Korea, to assist them in accurately estimating the number of crew members a ship will need.


They're currently building new ships for certain customers, and they'd like you to create a model and utilize it to estimate how many crew members the ships will require.


Metadata:
1. Measurements of ship size 
2. capacity 
3. crew 
4. age for 158 cruise ships.

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Crew_Members_Predictor').getOrCreate()

It is saved in a csv file for you called "ITI_data.csv". our task is to develop a regression model that will assist in predicting the number of crew members required for future ships. The client also indicated that they have found that particular cruise lines will differ in acceptable crew counts, thus this is most likely an important factor to consider when conducting your investigation.

In [2]:
df = spark.read.csv('ITI_data.csv',header=True,inferSchema=True)
df.show(5)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 5 rows



In [15]:
from pyspark.sql.functions import countDistinct
df.select(countDistinct("Cruise_line")).show()

+---------------------------+
|count(DISTINCT Cruise_line)|
+---------------------------+
|                         20|
+---------------------------+



In [16]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [17]:
df.count()

158

### OneHotEncoder 


In [18]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler

In [19]:
categoricalCols = [field for (field, dataType) in df.dtypes
                   if dataType == "string" and field !='Ship_name']
categoricalCols

['Cruise_line']

In [20]:
indexOutputCols = [x + "_Index" for x in categoricalCols]
indexOutputCols

['Cruise_line_Index']

In [21]:
oheOutputCols = [x + "_OHE" for x in categoricalCols]
oheOutputCols

['Cruise_line_OHE']

In [22]:
stringIndexer = StringIndexer(inputCols=categoricalCols,
                             outputCols=indexOutputCols,
                             handleInvalid='skip')
oheEncoder = OneHotEncoder(inputCols=indexOutputCols,
                          outputCols=oheOutputCols)

In [23]:
numericCols = [field for (field,dataType) in df.dtypes
              if ((dataType=='double')& (field!='crew'))]
numericCols

['Tonnage', 'passengers', 'length', 'cabins', 'passenger_density']

In [24]:
assemblerInputs = oheOutputCols + numericCols
assemblerInputs

['Cruise_line_OHE',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density']

###Use VectorAssembler to merge all columns into one column:

In [27]:
vecAssembler = VectorAssembler(inputCols=assemblerInputs,outputCol='features')

VectorAssembler_d58303026d07

### Create a Linear Regression Model 

In [26]:
from pyspark.ml.regression import LinearRegression

lr = LinearRegression(featuresCol='features',labelCol='crew')

# Split data

In [28]:
X_train, X_test = df.randomSplit([0.8,0.2],seed=0)

### Creating a Pipeline

In [29]:
# Building the pipeline
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[stringIndexer,oheEncoder,vecAssembler,lr])
pipelineModel = pipeline.fit(X_train)

In [30]:
predDF = pipelineModel.transform(X_test)

In [31]:
predDF.show()

+-------------+----------------+---+------------------+----------+------+------+-----------------+-----+-----------------+---------------+--------------------+------------------+
|    Ship_name|     Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density| crew|Cruise_line_Index|Cruise_line_OHE|            features|        prediction|
+-------------+----------------+---+------------------+----------+------+------+-----------------+-----+-----------------+---------------+--------------------+------------------+
|         Dawn|       Norwegian| 11|              90.0|      22.4|  9.65|  11.2|            40.18| 11.0|              4.0| (19,[4],[1.0])|(24,[4,19,20,21,2...|10.712117947988038|
|         Dawn|        Princess| 16|            77.499|      19.5|  8.56|  10.5|            39.74|  9.0|              2.0| (19,[2],[1.0])|(24,[2,19,20,21,2...| 9.103494234997342|
|      Empress| Royal_Caribbean| 23|            48.563|      20.2|  6.92|   8.0|            24.04| 6.71| 

### Model Evaluation

In [32]:
# Using RMSE
from pyspark.ml.evaluation import RegressionEvaluator

regressionEvaluator = RegressionEvaluator(predictionCol='prediction',
                                         labelCol='crew',
                                         metricName='rmse')

rmse = regressionEvaluator.evaluate(predDF)
#print("RMSE is {:.1f}".format(rmse))
print(f"RMSE is {rmse:.1f}")

# Using R^2
r2 = RegressionEvaluator(predictionCol='prediction',
                                         labelCol='crew',
                                         metricName='r2').evaluate(predDF)
print(f"R2 is {r2}")

RMSE is 0.7
R2 is 0.9528590212289405
